In [1]:
from inventory import Inventory
from exception import InventoryRemoveZeroError, InventoryRemoveExcessiveError
from entity import Entity
from data.sql import select_inventory, select_entity
from data.duck import dd, sql_inventory
from uuid import uuid4

In [2]:
# try adding a yellow chair to the lounge
item = Entity(description="Yellow Chair")
print(item)
location = Entity(description="Lounge")
print(location)

Yellow Chair (total inventory: 3) (4d10faad-8a10-436b-b732-011a7b4ad043)
Lounge (total inventory: 1) (03dd5c43-d66e-4c3f-a6eb-eb67b47c0373)


In [3]:
# check the inventory for the item
print(f"{item.description} inventory:")
dd.sql(sql_inventory(entity_id_child=item.entity_id)).show()
# and the inventory for the location
print(f"{location.description} inventory:")
dd.sql(sql_inventory(entity_id_parent=location.entity_id)).show()

yellow chair inventory:
┌───────┬───────┬──────────────┬─────────┬─────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │   parent    │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │   varchar   │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     3 │ yellow chair │ IN      │ dining room │ 4e18e23a-ca8b-4ed3-9ba6-8e2fb46d6cd8 │ 6dc856f4-42fe-4fe0-868b-9d00

In [4]:
# try adding 4 yellow chairs to the lounge
inventory = Inventory(entity_parent=location, entity_child=item, quantity=4)
print(inventory)
dd.sql(sql_inventory(entity_id_parent=location.entity_id,entity_id_child=item.entity_id)).show()

Inventory created successfully
4 yellow chairs (4d10faad-8a10-436b-b732-011a7b4ad043) in lounge (03dd5c43-d66e-4c3f-a6eb-eb67b47c0373)
┌───────┬───────┬──────────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     4 │ yellow chai

In [5]:
# check the inventory for the item
print(f"{item.description} inventory:")
dd.sql(sql_inventory(entity_id_child=item.entity_id)).show()
# and the inventory for the location
print(f"{location.description} inventory:")
dd.sql(sql_inventory(entity_id_parent=location.entity_id)).show()

yellow chair inventory:
┌───────┬───────┬──────────────┬─────────┬─────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │   parent    │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │   varchar   │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     4 │ yellow chair │ IN      │ lounge      │ fa792888-7f85-4e5b-9e8b-7093732a9ecc │ 8ce8d635-c5c5-49d7-bdcf-2b5c

In [6]:
# try adding a further 2 yellow chairs to the lounge
inventory = Inventory(entity_parent=location, entity_child=item, quantity=2)
print(inventory)
dd.sql(sql_inventory(entity_id_parent=location.entity_id,entity_id_child=item.entity_id)).show()

Inventory updated successfully
6 yellow chairs (4d10faad-8a10-436b-b732-011a7b4ad043) in lounge (03dd5c43-d66e-4c3f-a6eb-eb67b47c0373)
┌───────┬───────┬──────────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │    child     │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │   varchar    │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼──────────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     6 │ yellow chai

In [7]:
# try removing too many yellow chairs from the lounge
try:
    inventory = Inventory(entity_parent=location, entity_child=item, quantity=99999, action="REMOVE")
except InventoryRemoveExcessiveError as e:
    print(e)

There are not enough yellow chairs in lounge to remove


In [8]:
# remove all yellow chairs from lounge
new_quantity = inventory._quantity_new
inventory = Inventory(entity_parent=location, entity_child=item, quantity=new_quantity, action="REMOVE")
# inventory record should have been deleted
dd.sql(sql_inventory(entity_id_parent=location.entity_id,entity_id_child=item.entity_id)).show()

Inventory deleted successfully
┌───────┬───────┬─────────┬─────────┬─────────┬──────────────┬─────────────────────┬─────────┬─────────┐
│  lvl  │  qty  │  child  │   pos   │ parent  │ inventory_id │ inventory_id_parent │ parent  │  child  │
│ int64 │ int64 │ varchar │ varchar │ varchar │   varchar    │       varchar       │ varchar │ varchar │
├───────┴───────┴─────────┴─────────┴─────────┴──────────────┴─────────────────────┴─────────┴─────────┤
│                                                0 rows                                                │
└──────────────────────────────────────────────────────────────────────────────────────────────────────┘



In [9]:
# try removing a further yellow chair from the lounge
try:
    inventory = Inventory(entity_parent=location, entity_child=item, quantity=1, action="REMOVE")
except InventoryRemoveZeroError as e:
    print(e)

There are no yellow chairs in lounge to remove


In [10]:
# add a new level 1 location
location1 = Entity(description="Stillwaters",base_hierarchy_level = 1, entity_id="339cfbbb-f263-41f0-b4dc-5abcfe83e98c")
dd.sql(f"select * from entity_sql where entity_key = 'stillwater'").show()

Entity created successfully: Stillwaters (total inventory: 0) (339cfbbb-f263-41f0-b4dc-5abcfe83e98c)
┌──────────────────────────────────────┬──────────────────┬────────────┬────────────┬────────────┬──────────────────────┬────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │ entity_key │    noun    │ adjectives │ description_singular │ description_plural │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │  varchar   │  varchar   │  varchar   │       varchar        │      varchar       │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼────────────┼────────────┼────────────┼──────────────────────┼────────────────────┼──────────────────────┼─────────────────────┤
│ 339cfbbb-f263-41f0-b4dc-5abcfe83e98c │ None             │ stillwater │ stillwater │            │ stillwaters          │ stillwaters        │     

In [11]:
# add a new location
location2 = Entity(description="Dungeon", entity_id="9388641c-36bb-42f8-92b5-0d267098e7f1")
dd.sql(f"select * from entity_sql where entity_key = 'dungeon'").show()

Entity created successfully: Dungeon (total inventory: 0) (9388641c-36bb-42f8-92b5-0d267098e7f1)
┌──────────────────────────────────────┬──────────────────┬────────────┬─────────┬────────────┬──────────────────────┬────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │ entity_key │  noun   │ adjectives │ description_singular │ description_plural │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │  varchar   │ varchar │  varchar   │       varchar        │      varchar       │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼────────────┼─────────┼────────────┼──────────────────────┼────────────────────┼──────────────────────┼─────────────────────┤
│ 9388641c-36bb-42f8-92b5-0d267098e7f1 │ None             │ dungeon    │ dungeon │            │ dungeon              │ dungeons           │                    0 │ 

In [12]:
# create an inventory record for the new level 1 location
inventory = Inventory(entity_parent=location1, entity_child=location2, quantity=1)
print(location1.entity_id, location2.entity_id)
# inventory record newly created
dd.sql(sql_inventory(entity_id_parent=location1.entity_id,entity_id_child=location2.entity_id)).show()

Inventory created successfully
339cfbbb-f263-41f0-b4dc-5abcfe83e98c 9388641c-36bb-42f8-92b5-0d267098e7f1
┌───────┬───────┬─────────┬─────────┬─────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │  child  │   pos   │   parent    │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │ varchar │ varchar │   varchar   │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼─────────┼─────────┼─────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     1 │     1 │ dungeon │ IN      │ stillwaters │ 781435fc-f8

In [13]:
# add a cd rack to the dungeon
item = Entity("CD Rack")
location = Entity("Dungeon")
inventory = Inventory(entity_parent=location, entity_child=item, quantity=1)
# inventory record newly created
dd.sql(sql_inventory(entity_id_parent=location.entity_id,entity_id_child=item.entity_id)).show()

Entity created successfully: Cd Rack (total inventory: 0) (b6eb19a8-e85e-4ccf-a8a5-4830977b82f5)
Inventory created successfully
┌───────┬───────┬─────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │  child  │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │ varchar │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼─────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     2 │     1 │ cd rack │ IN      │ dungeon │ e5d29be7

In [14]:
# add a cd to the cd rack
item = Entity("The World Wont Listen by The Smiths")
location = Entity("CD Rack")
inventory = Inventory(entity_parent=location, entity_child=item, quantity=1)
# inventory record newly created
dd.sql(sql_inventory(entity_id_parent=location.entity_id,entity_id_child=item.entity_id)).show()

Entity created successfully: World Wont Listen By Smith (total inventory: 0) (f130174f-4e54-42b2-b85c-74ba85a466de)
Inventory created successfully
┌───────┬───────┬────────────────────────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │           child            │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │          varchar           │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼────────────────────────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼─

In [15]:
# add a lounge to the stillwaters location
item = Entity("lounge")
location = Entity("stillwaters")
inventory = Inventory(entity_parent=location, entity_child=item, quantity=1)
dd.sql(f"select * from entity_sql where entity_key = 'lounge'").show()
dd.sql(sql_inventory(entity_id_parent=location.entity_id,entity_id_child=item.entity_id)).show()

Inventory created successfully
┌──────────────────────────────────────┬──────────────────┬────────────┬─────────┬────────────┬──────────────────────┬────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │ entity_key │  noun   │ adjectives │ description_singular │ description_plural │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │  varchar   │ varchar │  varchar   │       varchar        │      varchar       │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼────────────┼─────────┼────────────┼──────────────────────┼────────────────────┼──────────────────────┼─────────────────────┤
│ 03dd5c43-d66e-4c3f-a6eb-eb67b47c0373 │ None             │ lounge     │ lounge  │            │ lounge               │ lounges            │                    0 │ 2025-03-15 09:16:13 │
└──────────────────────────────────────┴────

In [16]:
# add a sofa to the lounge
item = Entity("sofa")
location = Entity("lounge")
inventory = Inventory(entity_parent=location, entity_child=item, quantity=1)
dd.sql(f"select * from entity_sql where entity_key = 'sofa'").show()
dd.sql(sql_inventory(entity_id_parent=location.entity_id,entity_id_child=item.entity_id)).show()

Entity created successfully: Sofa (total inventory: 0) (2f74fb55-7cda-4dda-9553-69783b6dd93e)
Multiple parent inventories found:
1) lounge in st chads vicarage
2) lounge in stillwaters


Enter the number of the parent inventory to use:  2


Inventory created successfully
┌──────────────────────────────────────┬──────────────────┬────────────┬─────────┬────────────┬──────────────────────┬────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │ entity_key │  noun   │ adjectives │ description_singular │ description_plural │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │  varchar   │ varchar │  varchar   │       varchar        │      varchar       │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼────────────┼─────────┼────────────┼──────────────────────┼────────────────────┼──────────────────────┼─────────────────────┤
│ 2f74fb55-7cda-4dda-9553-69783b6dd93e │ None             │ sofa       │ sofa    │            │ sofa                 │ sofas              │                    0 │ 2025-03-15 09:16:40 │
└──────────────────────────────────────┴────

In [17]:
dd.close()